In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "features.csv"

# Load the latest version
df1 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "aslanahmedov/walmart-sales-forecast",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)



<ipython-input-2-bcf1b07f3c41>:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df1 = kagglehub.load_dataset(


In [ ]:
# Set the path to the file you'd like to load
file_path = "stores.csv"

# Load the latest version
df2 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "aslanahmedov/walmart-sales-forecast",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)


<ipython-input-3-92f671d8c215>:5: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df2 = kagglehub.load_dataset(


In [ ]:
# Set the path to the file you'd like to load
file_path = "train.csv"

# Load the latest version
df3 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "aslanahmedov/walmart-sales-forecast",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print(df3.shape)

<ipython-input-4-48e1ed0aa2cf>:5: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df3 = kagglehub.load_dataset(


(421570, 5)


In [ ]:
import pandas as pd

result = pd.merge(df1, df2, on='Store', how='inner')

In [ ]:
final = pd.merge(result, df3, on=['Store', 'Date'], how='inner')

In [ ]:

missing_percentages = final.isnull().sum() * 100 / len(final)
missing_percentages


,0
Store,0.000000
Date,0.000000
Temperature,0.000000
Fuel_Price,0.000000
MarkDown1,64.257181
MarkDown2,73.611025
MarkDown3,67.480845
MarkDown4,67.984676
MarkDown5,64.079038
CPI,0.000000


In [ ]:
final.drop(columns=['MarkDown1', 'MarkDown2', 'MarkDown3' , 'MarkDown4', 'MarkDown5'], inplace=True)

In [ ]:
final.drop(columns=['IsHoliday_y'], inplace=True)

In [ ]:
final.columns

Index(['Store', 'Date', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
       'IsHoliday_x', 'Type', 'Size', 'Dept', 'Weekly_Sales'],
      dtype='object')

In [ ]:
final.head(5)

,Store,ds,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday_x,Type,Size,Dept,y
0,1,2010-02-05,42.31,2.572,211.096358,8.106,False,A,151315,1,24924.50
1,1,2010-02-05,42.31,2.572,211.096358,8.106,False,A,151315,2,50605.27
2,1,2010-02-05,42.31,2.572,211.096358,8.106,False,A,151315,3,13740.12
3,1,2010-02-05,42.31,2.572,211.096358,8.106,False,A,151315,4,39954.04
4,1,2010-02-05,42.31,2.572,211.096358,8.106,False,A,151315,5,32229.38


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize encoder
encoder = LabelEncoder()

# Fit and transform
final['Type'] = encoder.fit_transform(final['Type'])

final['IsHoliday_x'] = encoder.fit_transform(final['IsHoliday_x'])



In [ ]:
#Data Preprocessing

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

y=final['Date']

final.drop(columns=['Date'], inplace=True)

scaled_data = scaler.fit_transform(final)

scaled_df = pd.DataFrame(scaled_data, columns=final.columns, index=final.index)

scaled_df['Date'] = y

scaled_df['Date'] = pd.to_datetime(scaled_df['Date'])

# Extract numerical features
scaled_df['hour'] = scaled_df['Date'].dt.hour
scaled_df['day'] = scaled_df['Date'].dt.day
scaled_df['dayofweek'] = scaled_df['Date'].dt.dayofweek
scaled_df['month'] = scaled_df['Date'].dt.month

scaled_df.drop(columns=['Date'], inplace=True)

In [ ]:
tgt=scaled_df['Weekly_Sales']

scaled_df.drop(columns=['Weekly_Sales'], inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_df, tgt, test_size=0.2, random_state=42)


In [ ]:
X_train['tgt']=y_train

X_test['tgt']=y_test

In [ ]:
X_train.columns

Index(['Store', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
       'IsHoliday_x', 'Type', 'Size', 'Dept', 'hour', 'day', 'dayofweek',
       'month', 'tgt'],
      dtype='object')

In [ ]:
X_test_e = X_test[X_test['IsHoliday_x'] == 1]

y_test_e = X_test_e['tgt']


## LSTM Model

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [ ]:
# Encoder-Decoder (LSTM)
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.hidden_size = hidden_size

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        _, (hn, _) = self.lstm(x)
        return hn[-1]  # (batch, hidden_size)

class MLPDecoder(nn.Module):
    def __init__(self, input_size):
        super(MLPDecoder, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.out = nn.Linear(16, 1)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        return self.out(x)

# Wrapper model with dropout for uncertainty
class FullModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(FullModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        features = self.encoder(x)
        return self.decoder(features)

# Create synthetic sequence data: assuming sliding window of 28
seq_len = 7
num_features = X_train.shape[1]
train_sequences = []
train_targets = []

for i in range(len(X_train) - seq_len):
    seq = torch.tensor(X_train.iloc[i:i+seq_len].values, dtype=torch.float32)
    target = torch.tensor([y_train.iloc[i+seq_len]], dtype=torch.float32)  # wrap in list to keep dimension
    train_sequences.append(seq)
    train_targets.append(target)

# Now stack them into tensors
train_sequences = torch.stack(train_sequences)  # Shape: (num_sequences, seq_len, num_features)
train_targets = torch.stack(train_targets)      # Shape: (num_sequences, 1)
encoder = LSTMEncoder(input_size=num_features)
decoder = MLPDecoder(input_size=encoder.hidden_size)
model = FullModel(encoder, decoder)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# Train model
model.train()
epochs = 2
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(train_sequences)
    loss = loss_fn(output, train_targets)
    loss.backward()
    optimizer.step()
    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Monte Carlo Dropout (for uncertainty)
def predict_with_uncertainty(f_model, input_seq, n_iter=100):
    f_model.train()  # keep dropout active
    preds = [f_model(input_seq).detach().numpy() for _ in range(n_iter)]
    preds = np.array(preds)
    pred_mean = preds.mean(axis=0)
    pred_std = preds.std(axis=0)
    return pred_mean, pred_std

# Example: get prediction + uncertainty for last sequence in train
sample_seq = train_sequences[-1].unsqueeze(0)
mean_pred, std_pred = predict_with_uncertainty(model, sample_seq)
print("Predicted Sales:", mean_pred.flatten()[0])
print("Uncertainty (Std Dev):", std_pred.flatten()[0])

Epoch 0, Loss: 0.0245
Predicted Sales: 0.01678781
Uncertainty (Std Dev): 1.8626451e-09


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# --- Step 1: Create test sequences ---
test_sequences = []
test_targets = []

for i in range(len(X_test) - seq_len):
    seq = torch.tensor(X_test.iloc[i:i+seq_len].values, dtype=torch.float32)
    target = torch.tensor([y_test.iloc[i+seq_len]], dtype=torch.float32)
    test_sequences.append(seq)
    test_targets.append(target)

test_sequences = torch.stack(test_sequences)
test_targets = torch.stack(test_targets)

# --- Step 2: Make predictions ---
model.eval()
with torch.no_grad():
    test_preds = model(test_sequences)

# --- Step 3: Evaluate ---
mae = mean_absolute_error(test_targets.numpy(), test_preds.numpy())
rmse = np.sqrt(mean_squared_error(test_targets.numpy(), test_preds.numpy()))

print(f"Test MAE: {mae:.2f}")
print(f"Test RMSE: {rmse:.2f}")

Test MAE: 0.02
Test RMSE: 0.04
